In [9]:
 #PWM_DIFF_CALCULATOR.IPYNB - calculating a bound on the pulse interval to cap deviation from the mean in PWM control

 # source for diffusion coeff: https://journals.asm.org/doi/10.1128/jb.185.5.1485-1491.2003#T1

# PACKAGE IMPORTS
import numpy as np
import scipy
import jax
import jax.numpy as jnp
import jaxopt
import functools
import diffrax as diffrax
from diffrax import diffeqsolve, ODETerm, SaveAt, PIDController, SteadyStateEvent

# plotting and data handling
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import matplotlib as mpltlb

# miscellaneous
import time

# PACKAGE SETUP
# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

Loading BokehJS ...

In [10]:
# CALCULATOR DEFINITION

def T_alpha_calc(alpha,  # maximum deviation from assumed inducer level, % of pulse concentration u_max
            l,  # length of tubing between the valve and the microfluidic chip, cm
            V,  # volume of 1 cm of tubing, uL
            nu,  # flow rate of the medium through the tubing, uL/min
            D,  # diffusion coefficient of the inducer, cm^2/min
            r_alpha_calc    # function for finding r_alpha - depends on the bound we set
            ):
    # get the Lambda value
    Lambda = 0.5*np.pi*(alpha/100)

    # get the r_alpha value
    r_alpha = r_alpha_calc(Lambda)

    # get the T_alpha
    T_alpha = np.sqrt(-((np.pi**2) * l * (V**3) * D) /
                      ((nu**3) * np.log(r_alpha)))

    # return the calculated T_alpha
    return T_alpha

# algebraic calculation of r_alpha, based on a very lenient bound
def r_alpha_algebraic(Lambda):
    return Lambda/(Lambda+1)

# a tighter bound on r_alpha, based on a numerical solution
def r_alpha_numerical(Lambda, func_for_root):
    func_for_root_of_r_alpha = lambda r_alpha: func_for_root(r_alpha, Lambda)

    # define an interval to search for the root
    r_alpha_lower_bound = 0 # will have func<0
    r_alpha_upper_bound = 0.99 # we need to consider the (0,1) interval, but at 1 the function is infinite => get some value close to 1 with func>0
    while(func_for_root_of_r_alpha(r_alpha_upper_bound)<=0):   # scan values 1-1e(-n) until we find a value where func>0
        r_alpha_upper_bound = 1-0.1*(1-r_alpha_upper_bound)

    # find the root
    threshold_problem = jaxopt.Bisection(optimality_fun=func_for_root_of_r_alpha,
                                         lower=r_alpha_lower_bound, upper=r_alpha_upper_bound,
                                         maxiter=10000, tol=1e-18,
                                         check_bracket=False)
    r_alpha = (threshold_problem.run()).params

    return r_alpha

In [11]:
# BEST-ACHIEVABLE RESOLUTION FINDER

def alpha_best_calc(l,  # length of tubing between the valve and the microfluidic chip, cm
            V,  # volume of 1 cm of tubing, uL
            nu,  # flow rate of the medium through the tubing, uL/min
            D,  # diffusion coefficient of the inducer, cm^2/min
            r_alpha_calc,    # function for finding r_alpha - depends on the bound we set
            alpha_values = np.linspace(50, 0.25, 200)    # values of alpha to sweep through
            ):
    for i in range(0, len(alpha_values)):
        T_alpha = T_alpha_calc(alpha_values_fine[j], l, V, nu, D, r_alpha_calc)
        rho_alpha = 100*DT/T_alpha   # finest resolution, % of u_max
        if(rho_alpha<=alpha_values[i]):
            alpha_best = alpha_values[i]
        else:
            break
    return alpha_best

In [12]:
# DEFINE PARAMETERS

# get the volume of 1 cm of tubing, uL
internal_diameter_in = 1/32  # internal diameter of the tubing in inches
internal_radius_m = internal_diameter_in * 0.0254 / 2  # internal radius in metres
volume_per_cm_m3 = np.pi * (internal_radius_m**2) /100  # volume of 1 cm of tubing in m^3
V = volume_per_cm_m3 * 1e9  # volume of 1 cm of tubing in uL
print(f"Volume of 1 cm of tubing: {V} uL")

# set the flow rate of the medium through the tubing, uL/min
nu = 20  # uL/min - common cruise flow rate
print(f"Flow rate of the medium through the tubing: {nu} uL/min")

# set the diffusion coefficient of the inducer, cm^2/min
D_cm2_s = 8.8e-6  # cm^2/s - diffusion coefficient of inducer in water at 37C
D = D_cm2_s * 60  # diffusion coefficient in cm^2/min
print(f"Diffusion coefficient of the inducer: {D} cm^2/min")


Volume of 1 cm of tubing: 4.948315225561501 uL
Flow rate of the medium through the tubing: 20 uL/min
Diffusion coefficient of the inducer: 0.000528 cm^2/min


In [13]:
# CALCULATE T_ALPHA for given l values

# set the maximum deviation from the assumed inducer level, % of pulse concentration u_max
alpha = 5

# set the range of l values to consider
l_values = np.linspace(10, 100, 9)
control_delays = l_values*V/nu   # control delays associated with the tubing lengths

# set the alpha values to consider
alpha_values =[10, 5, 1]

# define how we calculate r_alpha
# func_for_root_r_alpha_numerical = lambda r_alpha, Lambda: (r_alpha+0.5*r_alpha**2/(1-r_alpha**2))-Lambda
# r_alpha_calc = lambda Lambda: r_alpha_numerical(Lambda,func_for_root_r_alpha_numerical)
r_alpha_calc = r_alpha_algebraic

# find the T_alpha values for the given l and alpha values
T_alphas=np.zeros((len(alpha_values), len(l_values)))    # initialise
for i in range(0, len(alpha_values)):
    for j in range(0, len(l_values)):
        T_alphas[i,j] = T_alpha_calc(alpha_values[i], l_values[j], V, nu, D, r_alpha_calc)*60


In [14]:
# PLOT T_ALPHAS

# the range of T_alpha values
T_alpha_range = (0.75*np.min(T_alphas), 1.25 * np.max(T_alphas))

# initialise
T_alpha_fig = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_range=(l_values[0], l_values[-1]),
    y_range=T_alpha_range,
    x_axis_label="Tubing length l, cm",
    y_axis_label="T_α%, s",
    y_axis_type="log",
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
T_alpha_fig.output_backend = "svg"

# plot the cat protein concentrations
for i in range(0, len(alpha_values)):
    T_alpha_fig.line(l_values,T_alphas[i,:], line_width=2, legend_label=f"T_{alpha_values[i]}%",
             line_color=bkpalettes.Plasma5[i])

# create an extra x range for plotting integrase protein concentrations
T_alpha_fig.extra_x_ranges = {"t~": bkmodels.Range1d(start=control_delays[0], end=control_delays[-1])}
T_alpha_fig.add_layout(bkmodels.LinearAxis(x_range_name="t~", axis_label="Control delay 𝜏, min"), 'above')  # add the alternative axis

# create an extra y range for seconds as time units
DT=0.5
T_alpha_fig.extra_y_ranges = {"s": bkmodels.Range1d(start=DT/T_alpha_range[0]*100, end=DT/T_alpha_range[1]*100)}
T_alpha_fig.add_layout(bkmodels.LinearAxis(y_range_name="s", axis_label="PWM resolution, % of u_max"), 'right')  # add the alternative axis

# add legend
T_alpha_fig.legend.location = "bottom"
T_alpha_fig.legend.orientation = "horizontal"
T_alpha_fig.legend.label_text_font_size="8pt"
T_alpha_fig.legend.glyph_width = 8
T_alpha_fig.legend.spacing = 10
T_alpha_fig.legend.padding = 2
T_alpha_fig.legend.margin = 2


# set fonts
T_alpha_fig.xaxis.axis_label_text_font_size = "8pt"
T_alpha_fig.xaxis.major_label_text_font_size = "8pt"
T_alpha_fig.yaxis.axis_label_text_font_size = "8pt"
T_alpha_fig.yaxis.major_label_text_font_size = "8pt"

# show plots
bkplot.show(T_alpha_fig)

In [15]:
# GET THE FINEST ACHIEVABLE PWM INPUT RESOLUTION

# set the minimum time between switchings
DT=1/120  # 1/120 min = 0.5 s

# set the fine grid of alpha values to consider
alpha_values_fine = np.logspace(np.log10(50), -1, 1000) # fine grid of alpha values

# for every l value, find minimum alpha for which DT/T_alpha<=alpha
alpha_bests = np.zeros(len(l_values))  # initialise
for i in range(0, len(l_values)):
    alpha_bests[i] = alpha_best_calc(l_values[i], V, nu, D, r_alpha_calc, alpha_values_fine)

In [16]:
# PLOT THE FINEST ACHIEVABLE PWM INPUT RESOLUTIONS

# numbers of inputs to consider
numbers_of_concs = [2, 3, 6, 12]

# initialise
alpha_best_fig = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_range=(l_values[0], l_values[-1]),
    x_axis_label="Tubing length l, cm",
    y_axis_label="Best resolution α_best, % of u_max",
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
alpha_best_fig.output_backend = "svg"

# plot the cat protein concentrations
for i in range(0, len(numbers_of_concs)):
    alpha_best_fig.line(l_values, alpha_bests/(numbers_of_concs[i]-1),
                        line_width=2, legend_label=f"{numbers_of_concs[i]}", line_color=bkpalettes.Plasma5[i])

# create an extra x range for plotting integrase protein concentrations
alpha_best_fig.extra_x_ranges = {"t~": bkmodels.Range1d(start=control_delays[0], end=control_delays[-1])}
alpha_best_fig.add_layout(bkmodels.LinearAxis(x_range_name="t~", axis_label="Control delay 𝜏, min"), 'above')  # add the alternative axis

# set fonts
alpha_best_fig.xaxis.axis_label_text_font_size = "8pt"
alpha_best_fig.xaxis.major_label_text_font_size = "8pt"
alpha_best_fig.yaxis.axis_label_text_font_size = "8pt"
alpha_best_fig.yaxis.major_label_text_font_size = "8pt"

# # add legend
alpha_best_fig.legend.location = "top_right"
alpha_best_fig.legend.title_text_font_size="8pt"
alpha_best_fig.legend.label_text_font_size="8pt"
alpha_best_fig.legend.title='Sources:'
alpha_best_fig.legend.glyph_width = 15
alpha_best_fig.legend.spacing = 2
alpha_best_fig.legend.padding = 2
alpha_best_fig.legend.margin = 2
alpha_best_fig.legend.label_text_align = 'center'

# show plots
bkplot.show(alpha_best_fig)
